# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123


Python None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298434191202                   -0.85    5.1         
  2   -8.300208345881       -2.75       -1.25    1.0    126ms
  3   -8.300436631471       -3.64       -1.89    2.0    146ms
  4   -8.300461431490       -4.61       -2.75    2.9    164ms
  5   -8.300464406324       -5.53       -3.18    3.1    246ms
  6   -8.300464524597       -6.93       -3.35    9.8    276ms
  7   -8.300464584609       -7.22       -3.50    1.5    146ms
  8   -8.300464617303       -7.49       -3.65    2.2    161ms
  9   -8.300464634540       -7.76       -3.81    1.6    160ms
 10   -8.300464641205       -8.18       -3.98    4.9    266ms
 11   -8.300464643674       -8.61       -4.28    2.8    248ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67335215233                   -0.70    5.5         
  2   -16.67777299726       -2.35       -1.15    1.0    419ms
  3   -16.67851265183       -3.13       -1.85    4.2    912ms
  4   -16.67891218642       -3.40       -2.37    3.6    469ms
  5   -16.67925554500       -3.46       -2.93    2.9    458ms
  6   -16.67928534230       -4.53       -3.45    5.2    564ms
  7   -16.67928614253       -6.10       -3.77    2.2    424ms
  8   -16.67928621387       -7.15       -4.40    2.1    363ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32621133093                   -0.56    7.2         
  2   -33.33287155803       -2.18       -1.00    1.0    1.22s
  3   -33.33412986461       -2.90       -1.73    3.9    1.51s
  4   -33.33428703622       -3.80       -2.64    2.8    1.30s
  5   -33.33565632870       -2.86       -2.31    6.6    2.00s
  6   -33.33694001872       -2.89       -2.51    9.5    1.80s
  7   -33.33694081926       -6.10       -2.53    1.0    1.17s
  8   -33.33686980712   +   -4.15       -2.43    1.1    1.17s
  9   -33.33683650335   +   -4.48       -2.36    1.2    1.16s
 10   -33.33683720093       -6.16       -2.36    1.0    1.14s
 11   -33.33685217072       -4.82       -2.39    1.0    1.16s
 12   -33.33685393802       -5.75      

When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298260496522                   -0.85    5.2         
  2   -8.300227501905       -2.71       -1.59    1.2    118ms
  3   -8.300428061245       -3.70       -2.51    2.2    145ms
  4   -8.300369659484   +   -4.23       -2.26    4.2    205ms
  5   -8.300463881591       -4.03       -3.37    1.1    132ms
  6   -8.300464566877       -6.16       -3.74    3.5    219ms
  7   -8.300464634414       -7.17       -4.19    1.2    123ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32566923996                   -0.56    6.4         
  2   -33.33074732209       -2.29       -1.28    1.0    989ms
  3   -30.37945535120   +    0.47       -0.83    4.9    1.67s
  4   -32.84627380727        0.39       -1.21    5.4    1.86s
  5   -33.27930132917       -0.36       -1.39    3.0    1.40s
  6   -33.12437625461   +   -0.81       -1.40    3.0    1.38s
  7   -33.22329612824       -1.00       -1.52    3.4    1.38s
  8   -33.33392716727       -0.96       -2.25    2.4    1.20s
  9   -33.33490426877       -3.01       -2.32    4.1    1.63s
 10   -33.33560671573       -3.15       -2.28    2.2    1.33s
 11   -33.33680800223       -2.92       -2.80    1.8    1.16s
 12   -33.33686737024       -4.23      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.